In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!pip install onnx onnxruntime onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install ultralytics
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [6]:
# Step 1: Export YOLOv8 model to ONNX format
from ultralytics import YOLO
import os
import onnx
import numpy as np
from onnxruntime.quantization import quantize_static, QuantType

# Load your model
model_path = "/content/drive/MyDrive/saved_model/best.pt"
model = YOLO(model_path)

# Export to ONNX format
onnx_path = model.export(format="onnx", simplify=True)
print(f"Model exported to ONNX: {onnx_path}")


Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
YOLOv8n-cls summary (fused): 30 layers, 1,454,095 parameters, 0 gradients, 3.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/saved_model/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 15) (2.9 MB)
requirements: Ultralytics requirement ['onnxslim'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 4.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.1s, installed 1 package: ['onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.50...
ONNX: export success ✅ 6.4s, saved as '/content/drive/MyDrive/saved_model/best.onnx' (5.6 MB)

Export complete (8.0s)
Results saved to /content/drive/MyDrive/saved_model
Predict:         yolo predict task=classify model=/content/drive/MyDrive/saved_model/best.onnx imgsz=224  
Valid

In [9]:

# Step 2: Prepare calibration data (required for static quantization)
# This is a simplified example - ideally use actual data from your dataset
def representative_data_gen():
    # Generate 20 sample images (random data in this example)
    for _ in range(20):
        # Create random input in the expected shape [1, 3, 640, 640]
        # Adjust the shape based on your model's input requirements
        input_shape = model.model.yaml.get('imgsz', 640)
        if isinstance(input_shape, int):
            input_shape = [input_shape, input_shape]
        sample = np.random.random((1, 3, input_shape[0], input_shape[1])).astype(np.float32)
        yield [sample]

In [10]:

# Step 3: Create a calibration dataset file for quantization
calibration_data_file = "calibration.npz"
calibration_data = []

print("Generating calibration data...")
for data in representative_data_gen():
    calibration_data.append(data[0])

np.savez(calibration_data_file, *calibration_data)
print(f"Calibration data saved to {calibration_data_file}")


Generating calibration data...
Calibration data saved to calibration.npz


In [15]:
# Step 4: Perform static quantization to INT8
print("Starting INT8 quantization...")
quantized_model_path = onnx_path.replace('.onnx', '_quantized_int8.onnx')

# Configure quantization options
quantize_static(
    model_input=onnx_path,
    model_output=quantized_model_path,
    calibration_data_path=calibration_file,
    quant_format=QuantType.QInt8,
    per_channel=False,
    weight_type=QuantType.QInt8,
)

print(f"INT8 quantized model saved to {quantized_model_path}")

Starting INT8 quantization...


NameError: name 'calibration_file' is not defined

In [14]:



# Step 5: Compare model sizes
original_size = os.path.getsize(onnx_path) / (1024 * 1024)
quantized_size = os.path.getsize(quantized_model_path) / (1024 * 1024)

print(f"Original ONNX model size: {original_size:.2f} MB")
print(f"Quantized INT8 model size: {quantized_size:.2f} MB")
print(f"Reduction: {(1 - quantized_size/original_size) * 100:.1f}%")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/saved_model/best_quantized_int8.onnx'

In [ ]:



# Optional: If you want to try TFLite as well
# Step 6: Export to TFLite format with INT8 quantization
tf_path = model.export(format="tflite", int8=True)
print(f"Model exported to TFLite with INT8 quantization: {tf_path}")

tflite_size = os.path.getsize(tf_path) / (1024 * 1024)
print(f"TFLite INT8 model size: {tflite_size:.2f} MB")
print(f"Reduction from original: {(1 - tflite_size/original_size) * 100:.1f}%")

In [17]:
# Simplified approach focusing on TFLite export which has more stable API
from ultralytics import YOLO
import os

# Step 1: Load your model
model_path = "/content/drive/MyDrive/saved_model/best.pt"
model = YOLO(model_path)

print("Model loaded successfully!")

# First approach: Try TFLite with INT8 quantization
# This is often the easiest and most reliable approach
try:
    print("\nExporting to TFLite with INT8 quantization...")
    tf_path = model.export(format="tflite", int8=True)
    print(f"✅ Success! Model exported to: {tf_path}")

    original_size = os.path.getsize(model_path) / (1024 * 1024)
    tflite_size = os.path.getsize(tf_path) / (1024 * 1024)
    print(f"Original model size: {original_size:.2f} MB")
    print(f"TFLite INT8 model size: {tflite_size:.2f} MB")
    print(f"Reduction: {(1 - tflite_size/original_size) * 100:.1f}%")
except Exception as e:
    print(f"❌ TFLite export failed: {e}")
    print("You may need to install tensorflow: pip install tensorflow")

# Second approach: Alternative ONNX approach with simpler quantization
try:
    print("\nExporting to ONNX format...")
    onnx_path = model.export(format="onnx", simplify=True)
    print(f"✅ Success! ONNX model exported to: {onnx_path}")

    # Manual ONNX INT8 conversion using the correct current API
    import onnx
    from onnxruntime.quantization import quantize_dynamic, QuantType

    print("\nApplying dynamic quantization to ONNX model...")
    quantized_model_path = onnx_path.replace('.onnx', '_quantized_int8.onnx')

    # Dynamic quantization is more reliable and doesn't need calibration data
    quantize_dynamic(
        model_input=onnx_path,
        model_output=quantized_model_path,
        weight_type=QuantType.QInt8
    )

    print(f"✅ Success! Quantized model saved to: {quantized_model_path}")

    onnx_size = os.path.getsize(onnx_path) / (1024 * 1024)
    quantized_size = os.path.getsize(quantized_model_path) / (1024 * 1024)
    print(f"ONNX model size: {onnx_size:.2f} MB")
    print(f"Quantized ONNX model size: {quantized_size:.2f} MB")
    print(f"Reduction: {(1 - quantized_size/onnx_size) * 100:.1f}%")
except Exception as e:
    print(f"❌ ONNX quantization failed: {e}")
    print("Try installing: pip install onnx onnxruntime")

# For debugging
import sys
print(f"\nPython version: {sys.version}")
try:
    import onnxruntime
    print(f"ONNXRuntime version: {onnxruntime.__version__}")
except:
    print("ONNXRuntime not installed")

try:
    import tensorflow
    print(f"TensorFlow version: {tensorflow.__version__}")
except:
    print("TensorFlow not installed")

Model loaded successfully!

Exporting to TFLite with INT8 quantization...
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=imagenet10'.
YOLOv8n-cls summary (fused): 30 layers, 1,454,095 parameters, 0 gradients, 3.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/saved_model/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 15) (2.9 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx2tf>=1.26.3', 'tflite_support'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 213.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

100%|██████████| 1.11M/1.11M [00:00<00:00, 5.24MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 58.88file/s]


ONNX: starting export with onnx 1.17.0 opset 19...


ONNX: slimming with onnxslim 0.1.50...
ONNX: export success ✅ 0.4s, saved as '/content/drive/MyDrive/saved_model/best.onnx' (5.6 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=imagenet10'

Dataset not found ⚠️, missing path /content/datasets/imagenet10, attempting download...


100%|██████████| 71.1k/71.1k [00:00<00:00, 803kB/s]
Unzipping /content/datasets/imagenet10.zip to /content/datasets/imagenet10...: 100%|██████████| 24/24 [00:00<00:00, 4303.69file/s]

Dataset download success ✅ (1.5s), saved to /content/datasets/imagenet10

train: /content/datasets/imagenet10/train... found 12 images in 10 classes ✅ 
val: /content/datasets/imagenet10/val... found 12 images in 10 classes ✅ 
test: None...



Scanning /content/datasets/imagenet10/val/n01440764... 0 images, 12 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<00:00, 247.43it/s]

WARNING ⚠️ No labels found in /content/datasets/imagenet10/val/n01440764.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
New cache created: /content/datasets/imagenet10/val/n01440764.cache
WARNING ⚠️ No labels found in /content/datasets/imagenet10/val/n01440764.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
TensorFlow SavedModel: WARNING ⚠️ >300 images recommended for INT8 calibration, found 12 images.


TensorFlow SavedModel: starting TFLite export with onnx2tf 1.27.2...
Saved artifact at '/content/drive/MyDrive/saved_model/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 224, 224, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 15), dtype=tf.float32, name=None)
Captures:
  140094394866128: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  140094394865744: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  140094394866512: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  140094394869776: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  140094394867664: TensorSpec(shape=(3, 3, 16, 32), dtype=tf.float32, name=None)
  140094394870736: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  140094394870928: TensorSpec(shape=(1, 1, 32, 32), dtype=tf.float32, name=None)
  140094394870160: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  140094394

✅ Success! ONNX model exported to: /content/drive/MyDrive/saved_model/best.onnx

Applying dynamic quantization to ONNX model...
✅ Success! Quantized model saved to: /content/drive/MyDrive/saved_model/best_quantized_int8.onnx
ONNX model size: 5.57 MB
Quantized ONNX model size: 1.45 MB
Reduction: 73.9%

Python version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
ONNXRuntime version: 1.21.0
TensorFlow version: 2.18.0
